In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
X_train = np.load("vectors/X_train.npy")
Y_train = np.load("vectors/Y_train.npy")

In [3]:
X_valid = np.load("vectors/X_valid.npy")
Y_valid = np.load("vectors/Y_valid.npy")

In [4]:
X_test = np.load("vectors/X_test.npy")
Y_test = np.load("vectors/Y_test.npy")

In [5]:
maxlen = X_train.shape[1]
vocab_size = 26000
embeding_dimension = 10
number_of_heads = 6 

In [6]:
norm_layer = layers.LayerNormalization(epsilon=1e-6)

In [7]:
dropout = layers.Dropout(0.15)

In [8]:
feednn = keras.Sequential(
    [
        layers.Dense(10, activation="relu"),
        layers.Dense(embeding_dimension),
    ]
)

2022-01-13 16:56:59.605013: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
mha = layers.MultiHeadAttention(num_heads = number_of_heads, key_dim = embeding_dimension)

In [10]:
token_embending = layers.Embedding(input_dim=vocab_size, output_dim=embeding_dimension)

In [11]:
positional_embending = layers.Embedding(input_dim=maxlen, output_dim=embeding_dimension)

In [12]:
inputs = layers.Input(shape=(maxlen,))

pos = tf.range(start=0, limit=tf.shape(inputs)[-1], delta=1)
pos = positional_embending(pos)
x = token_embending(inputs)
x += pos

a = mha(x, x)
a = dropout(a)
a = norm_layer(x + a)
f = feednn(a)
f = dropout(f)
x = layers.GlobalAveragePooling1D()(f)
x = layers.Dense(20, activation="relu")(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), batch_size=16, epochs=3)

Epoch 1/3
468/468 [==============================] - 18s 36ms/step - loss: 0.5663 - accuracy: 0.7373 - val_loss: 0.5318 - val_accuracy: 0.7438
Epoch 2/3
468/468 [==============================] - 19s 41ms/step - loss: 0.3826 - accuracy: 0.8317 - val_loss: 0.2950 - val_accuracy: 0.8882
Epoch 3/3
468/468 [==============================] - 24s 51ms/step - loss: 0.2240 - accuracy: 0.9097 - val_loss: 0.1795 - val_accuracy: 0.9318


In [13]:
model.evaluate(X_test, Y_test, batch_size=32)

56/56 [==============================] - 1s 24ms/step - loss: 0.7028 - accuracy: 0.7042


[0.7027730345726013, 0.7042253613471985]